In [ ]:
pip install -r Project1/requirements.txt

In [2]:
import yfinance as yf
import plotly.graph_objs as go
import pandas as pd

# Data downloading

In [3]:
def get_data(instrument: str,
             start_date: str,
             end_date: str,
             interval: str) -> pd.DataFrame:
    """
        Fetch historical market data from Yahoo Finance for a given instrument between the provided start and end dates at the given interval.
        The function returns a cleaned DataFrame with the data, excluding any missing values.

        Parameters:
        instrument (str): The ticker symbol of the instrument e.g. 'MSFT'.
        start_date (str): The start date for the data in 'YYYY-MM-DD' format.
        end_date (str): The end date for the data in 'YYYY-MM-DD' format.
        interval (str): The time interval between data points. Valid intervals are: ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo'].

        Returns:
        pd.DataFrame: A Pandas DataFrame containing the historical market data for the given instrument, with any rows containing missing values (NaN) removed.
    """
    
    data = yf.download(tickers=instrument,
                       start=start_date,
                       end=end_date,
                       interval=interval)
    
    data = data.dropna(how='any')

    return data

# Indicators

## MACD

In [4]:
def macd(data: pd.DataFrame) -> pd.DataFrame:
    """
        Calculate the MACD (Moving Average Convergence Divergence) and generate buy/sell signals.

        Parameters:
        data (pd.DataFrame): DataFrame containing historical market data.

        Returns:
        pd.DataFrame: DataFrame with additional columns for 'MACD Buy Signal' and 'MACD Sell Signal', where a 1 indicates a signal and 0 means no signal.
    """

    data = data.copy()

    data['EMA12'] = data['Adj Close'].ewm(span=12, adjust=False).mean()
    data['EMA26'] = data['Adj Close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = data['EMA12'] - data['EMA26'] # MACD Line
    data['Signal Line'] = data['MACD'].ewm(span=9, adjust=False).mean() # Signal Line

    data['MACD Buy Signal'] = 0
    data['MACD Sell Signal'] = 0

    for i in range(1, len(data)):
        # buy signal: MACD crosses above Signal Line
        if data['MACD'].iloc[i] > data['Signal Line'].iloc[i] and data['MACD'].iloc[i-1] <= data['Signal Line'].iloc[i-1]:
            data.loc[data.index[i], 'MACD Buy Signal'] = 1
        # sell signal: MACD crosses below Signal Line
        elif data['MACD'].iloc[i] < data['Signal Line'].iloc[i] and data['MACD'].iloc[i-1] >= data['Signal Line'].iloc[i-1]:
            data.loc[data.index[i], 'MACD Sell Signal'] = 1

    # drop intermediate columns
    data = data.drop(columns=['EMA12', 'EMA26', 'MACD', 'Signal Line'])

    return data

In [5]:
strategies = {
    'MACD': macd
}

In [6]:
def run_strategy(data: pd.DataFrame, strategy: str) -> pd.DataFrame:
    """
        Executes the specified trading strategy on the given data.

        Parameters:
        data (pd.DataFrame): DataFrame containing historical market data on which the strategy will be applied.
                             The DataFrame must have relevant columns for the selected strategy.
        
        strategy (str): The name of the strategy to be applied.

        Returns:
        pd.DataFrame: DataFrame with the strategy applied, including any newly added columns like buy/sell signals.
    """
    
    if strategy in strategies:
        data = strategies[strategy](data=data)
    else:
        print("Invalid strategy selected!")

    return data

# Plots

## Buy and sell signal plot

In [7]:
def plot_buy_sell_signal(data: pd.DataFrame,
                         instrument: str,
                         buy_signal_column: str,
                         sell_signal_column: str,
                         title: str):
    """
        Plots a price chart with buy and sell signals marked on it.

        Parameters:
        data (pd.DataFrame): A DataFrame containing historical market data. The DataFrame must have
                            at least the 'Adj Close' column and the specified buy/sell signal columns.
        
        instrument (str): The ticker symbol or name of the instrument being plotted.

        buy_signal_column (str): The column name containing the buy signals (e.g., 'MACD Buy Signal').
                                The function will mark buy signals where the value in this column is 1.

        sell_signal_column (str): The column name containing the sell signals (e.g., 'MACD Sell Signal').
                                The function will mark sell signals where the value in this column is 1.
        
        title (str): The title of the plot, which will be displayed on the chart after instrument name.
    """
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['Adj Close'],
        mode='lines',
        name='Price',
        line=dict(color='blue')
    ))

    # add buy signals to the price chart
    buy_signals = data[data[buy_signal_column] == 1].index
    fig.add_trace(go.Scatter(
        x=buy_signals,
        y=data.loc[buy_signals, 'Adj Close'],
        mode='markers',
        name=buy_signal_column,
        marker=dict(color='green', symbol='triangle-up', size=10)
    ))

    # add sell signals to the price chart
    sell_signals = data[data[sell_signal_column] == 1].index
    fig.add_trace(go.Scatter(
        x=sell_signals,
        y=data.loc[sell_signals, 'Adj Close'],
        mode='markers',
        name=sell_signal_column,
        marker=dict(color='red', symbol='triangle-down', size=10)
    ))

    fig.update_layout(
        title=f"{instrument} - {title}",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        xaxis_rangeslider_visible=False,
        hovermode="x unified",
    )

    fig.show()

# Performance Evaluation

## Win Rate

In [8]:
def calculate_win_rate(data: pd.DataFrame, 
                       buy_signal_column: str, 
                       sell_signal_column: str) -> float:
    """
        Calculates the win rate of a trading strategy based on buy and sell signals in the given data.

        This function simulates a trading strategy where a position is opened on a buy signal
        and closed on a sell signal assuming one signal at the time (if buy signal then wait for sell signal).

        Parameters:
        data (pd.DataFrame): A DataFrame containing market data, including the 'Adj Close' column
                            and the buy/sell signal columns specified by `buy_signal_column` and `sell_signal_column`.
        
        buy_signal_column (str): The name of the column containing buy signals (e.g., 'MACD Buy Signal').
                                A value of 1 in this column indicates a buy signal.
        
        sell_signal_column (str): The name of the column containing sell signals (e.g., 'MACD Sell Signal').
                                A value of 1 in this column indicates a sell signal.
    """

    open_position = False  # track if there is an open position
    total_trades = 0       # total number of trades (both wins and losses)
    wins = 0               # total number of winning trades

    for i in range(1, len(data)):
        if data[buy_signal_column].iloc[i] == 1 and not open_position: # buy
            open_position = True # open the position
            entry_price = data['Adj Close'].iloc[i]
            print(f"Buying  at {data.index[i]}: {entry_price:.2f}")
        elif data[sell_signal_column].iloc[i] == 1 and open_position: # sell
            exit_price = data['Adj Close'].iloc[i]
            profit_loss = exit_price - entry_price
            profit_loss_percent = (profit_loss / entry_price) * 100
            total_trades += 1  # increment total trades
            if profit_loss > 0: 
                wins += 1 # increment total wins
                print(f"Selling at {data.index[i]}: {exit_price:.2f} | Profit: {profit_loss_percent:.2f}%")
            else:
                print(f"Selling at {data.index[i]}: {exit_price:.2f} | Loss: {profit_loss_percent:.2f}%")
            open_position = False  # close the position
    
    if total_trades > 0:
        win_rate = wins / total_trades
    else:
        win_rate = 0

    win_rate = win_rate * 100

    print(f"Win Rate: {win_rate:.2f}%")


# Results

In [ ]:
instrument='AAPL'
data = get_data(instrument=instrument, start_date='2000-01-01', end_date='2024-11-01', interval='1mo')
data = run_strategy(data=data, strategy='MACD')
plot_buy_sell_signal(data=data, instrument=instrument, buy_signal_column='MACD Buy Signal', sell_signal_column='MACD Sell Signal', title = 'Price with MACD Buy/Sell Signals')
win_rate = calculate_win_rate(data=data, buy_signal_column='MACD Buy Signal', sell_signal_column='MACD Sell Signal')